In [1]:
# ────────────────────────────────────────────────────────────────────
# Colab IV vs K (Yahoo Finance)
# Διαδραστική ροή: Ticker → Type (Call/Put) → Επιλογή λήξης → Γράφημα
# ────────────────────────────────────────────────────────────────────

# Εγκατάσταση βιβλιοθηκών (ήσυχα)
!pip -q install yfinance ipywidgets

import warnings
warnings.filterwarnings("ignore")

import sys
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------ Helpers ------------------------

def get_spot_price(tkr_obj):
    """Επιστρέφει προσεγγιστική spot τιμή."""
    spot = None
    # Προσπάθησε fast_info
    try:
        fi = getattr(tkr_obj, "fast_info", None)
        if fi and "last_price" in fi and fi["last_price"]:
            spot = float(fi["last_price"])
    except Exception:
        spot = None
    # Εναλλακτικά από ιστορικά (τελευταίο close)
    if spot is None:
        try:
            hist = tkr_obj.history(period="1d", auto_adjust=False)
            if not hist.empty:
                spot = float(hist["Close"].iloc[-1])
        except Exception:
            spot = None
    return spot

def clean_chain(df):
    """Καθαρίζει option chain dataframe και επιστρέφει (strike, IV) ταξινομημένα."""
    if df is None or df.empty:
        return pd.DataFrame(columns=["strike", "IV_pct"])
    keep = df[["strike", "impliedVolatility"]].dropna()
    # IV σε δεκαδική μορφή (0.25 = 25%): κρατάμε λογικές τιμές
    keep = keep[(keep["impliedVolatility"] > 0.001) & (keep["impliedVolatility"] < 5.0)]
    keep = keep.sort_values("strike").copy()
    keep["IV_pct"] = keep["impliedVolatility"] * 100.0
    return keep[["strike", "IV_pct"]]

def plot_iv_vs_k(df_iv, spot, ticker, expiry, opt_type_label):
    """Ένα ενιαίο γράφημα (χωρίς subplots), μόνο matplotlib, χωρίς explicit colors."""
    if df_iv is None or df_iv.empty:
        print("Δεν υπάρχουν επαρκή δεδομένα IV για σχεδίαση.")
        return
    plt.figure(figsize=(9, 6))
    # scatter των σημείων IV vs Strike
    plt.scatter(df_iv["strike"], df_iv["IV_pct"], label=f"{opt_type_label}", alpha=0.85, s=28)
    # κατακόρυφη γραμμή στη spot
    if spot is not None:
        plt.axvline(spot, linestyle="--", linewidth=1.2, alpha=0.9, label=f"Spot ≈ {spot:.2f}")
    plt.title(f"{ticker} — Implied Volatility vs Strike (λήξη {expiry}, {opt_type_label})")
    plt.xlabel("Strike (K)")
    plt.ylabel("Implied Volatility (%)")
    plt.grid(True, which="both", linestyle=":", linewidth=0.7)
    plt.legend()
    plt.tight_layout()
    plt.show()

# ------------------------ Widgets ------------------------

ticker_w = widgets.Text(
    value="AAPL",
    placeholder="π.χ. AAPL",
    description="Ticker:",
    disabled=False
)

type_w = widgets.ToggleButtons(
    options=[("Call", "call"), ("Put", "put")],
    value="call",
    description="Type:",
    disabled=False,
    button_style=""
)

fetch_btn = widgets.Button(
    description="Φέρε διαθέσιμες λήξεις",
    icon="download",
    button_style=""
)

expiry_w = widgets.Dropdown(
    options=[],
    description="Λήξη:",
    disabled=True
)

plot_btn = widgets.Button(
    description="Σχεδίαση IV vs K",
    icon="line-chart",
    button_style="",
    disabled=True
)

out = widgets.Output()

# ------------------------ Handlers ------------------------

def on_fetch_clicked(b):
    with out:
        out.clear_output()
        ticker = ticker_w.value.strip().upper()
        if not ticker:
            print("🔎 Δώσε ένα έγκυρο ticker (π.χ. AAPL).")
            return
        try:
            tkr = yf.Ticker(ticker)
            expirations = list(tkr.options or [])
        except Exception as e:
            print(f"❌ Αποτυχία ανάκτησης λήξεων για {ticker}: {e}")
            return

        if not expirations:
            print(f"⚠️ Δεν βρέθηκαν διαθέσιμες λήξεις για {ticker}.")
            expiry_w.options = []
            expiry_w.disabled = True
            plot_btn.disabled = True
            return

        # Επιλογή προεπιλογής: η πιο κοντινή μελλοντική
        expiry_w.options = expirations
        expiry_w.value = expirations[0]
        expiry_w.disabled = False
        plot_btn.disabled = False

        print(f"✅ Βρέθηκαν {len(expirations)} λήξεις για {ticker}. Διάλεξε από το dropdown και πάτα 'Σχεδίαση'.")

def on_plot_clicked(b):
    with out:
        out.clear_output()
        ticker = ticker_w.value.strip().upper()
        opt_type = type_w.value  # "call" ή "put"
        expiry = expiry_w.value

        if not ticker:
            print("🔎 Δώσε ένα έγκυρο ticker (π.χ. AAPL).")
            return
        if not expiry:
            print("🔽 Διάλεξε ημερομηνία λήξης από το dropdown.")
            return

        print(f"⏳ Κατέβασμα option chain για {ticker}, λήξη {expiry}, τύπος {opt_type}...")
        try:
            tkr = yf.Ticker(ticker)
            chain = tkr.option_chain(expiry)
        except Exception as e:
            print(f"❌ Αποτυχία ανάκτησης option chain: {e}")
            return

        df = chain.calls if opt_type == "call" else chain.puts
        df_iv = clean_chain(df)
        spot = get_spot_price(tkr)

        n_pts = len(df_iv)
        if n_pts == 0:
            print("⚠️ Δεν υπάρχουν επαρκή δεδομένα IV για σχεδίαση.")
            return

        print(f"📈 Σημεία: {n_pts} | Spot: {('%.2f' % spot) if spot is not None else '—'}")
        plot_iv_vs_k(df_iv, spot, ticker, expiry, "Calls" if opt_type=="call" else "Puts")

def on_ticker_changed(change):
    # Αν αλλάξει το ticker, απαιτούμε ξανά 'Φέρε διαθέσιμες λήξεις'
    expiry_w.options = []
    expiry_w.disabled = True
    plot_btn.disabled = True

ticker_w.observe(on_ticker_changed, names="value")
fetch_btn.on_click(on_fetch_clicked)
plot_btn.on_click(on_plot_clicked)

# ------------------------ Layout ------------------------

ui = widgets.VBox([
    widgets.HBox([ticker_w, type_w]),
    widgets.HBox([fetch_btn]),
    widgets.HBox([expiry_w, plot_btn]),
    out
])

display(ui)

print("💡 Ροή χρήσης:")
print("1) Συμπλήρωσε Ticker και διάλεξε Type (Call/Put).")
print("2) Πάτησε «Φέρε διαθέσιμες λήξεις».")
print("3) Διάλεξε ημερομηνία στο πεδίο «Λήξη».")
print("4) Πάτησε «Σχεδίαση IV vs K» για το γράφημα.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.6 MB/s eta 0:00:00


💡 Ροή χρήσης:
1) Συμπλήρωσε Ticker και διάλεξε Type (Call/Put).
2) Πάτησε «Φέρε διαθέσιμες λήξεις».
3) Διάλεξε ημερομηνία στο πεδίο «Λήξη».
4) Πάτησε «Σχεδίαση IV vs K» για το γράφημα.
